<h1><span style="color:red">Generate a SuAVE survey from the UN SDG database</span></h1>

This Jupyter Notebook is designed to access SDG database and let users select variables and years to add to the survey file for visual analysis.

You can execute cells in sequence (by clicking 'Run' above, or pressing Shift-Enter). 
The source of the SDG database is: Statistics Division, Department of Economic and Social Affairs, United Nations.
The database can be downloaded from https://unstats.un.org/sdgs/indicators/database/

Definitions of indicators are available at the same page. The current version is October 10, 2018



## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
import xlrd
import csv
import urllib.request
import io
import os
from collections import OrderedDict
import re
import requests


## 2. Access local version of the UN SDG Database

In [ ]:
file_names = []
file_directory = "/lib-nfs/largedatasets"
for file in os.listdir(file_directory):
    if file.endswith('.csv'):
        file_names.append(os.path.join(file_directory, file))
    
def f(file_name):
    return file_name
print('The source data file should contain fields named TimePeriod, SeriesCode, Indicator, and GeoAreaCode.') 
print('GeoAreaCode values will be matched with "UN Code" values in the survey file (this can be changed below).')
print('Select the source data file:')
fileName = interact(f, file_name=file_names)

## 3. Prepare to query the database

In [ ]:
new_csv_file = fileName.widget.result

df = pd.read_csv(new_csv_file, low_memory=False)

time_periods = OrderedDict()
time_periods_list = sorted(list(set(list(df['TimePeriod']))))
for i in range(0, len(time_periods_list)-1):
    time_periods[str(time_periods_list[i])] = int(time_periods_list[i])


series_codes = []
for i in range(0, len(df['SeriesCode'])):
#     series_codes.append(df['SeriesCode'][i] + "-" + df['Indicator'][i])
    series_codes.append(df['Indicator'][i] + "-" + df['SeriesCode'][i])

series_codes = sorted(list(set(series_codes)))

## 4. Select years of data to retrieve

Use Shift or Ctrl to select multiple years

In [ ]:
def g(TimePeriod):
    return TimePeriod

time_period_widget = interact(g, TimePeriod=widgets.SelectMultiple(options=time_periods))

## 5. Select data series to retrieve

Use Shift or Ctrl to select multiple series

In [ ]:
def h(SeriesCode):
    return SeriesCode

series_code_widget = interact(h, SeriesCode=widgets.SelectMultiple(options=series_codes))

## 6. Read the SuAVE survey file with base SDG data

In [ ]:
old_df = panellibs.extract_data(absolutePath + csv_file)


## 7. Add the newly selected data series to the base SDG data

In [ ]:
series_code = series_code_widget.widget.result
time_period = time_period_widget.widget.result

for i in range(0, len(series_code)):
    for j in range(0, len(time_period)):
        series_code_array = series_code[i].split("-")
        series_code_0 = series_code_array[0]
        series_code_1 = series_code_array[1]
            
        rows = df.loc[(df['TimePeriod'] == time_period[j]) & 
                          (df['SeriesCode'] == series_code_1) &
                          (df['Indicator'] == series_code_0)]
        
        if (not rows.empty):
        
#             new_col_name = series_code[i] + "-" + str(time_period[j])
            new_col_name = series_code_0 + "-" + series_code_1 +"-" + str(time_period[j])+"#number"
            old_df[new_col_name] = np.nan
        
            for k in range(0, len(rows.index)):
                
                geo_code = rows.iloc[k]['GeoAreaCode']
                value = rows.iloc[k]['Value']
                
# IMPORTANT: depending on the survey version, you may need to specify a field
# to match country numeric IDs in the UN SDG database
# In the 2015 version of the SDG survey in SuAVE, this field is 'UN Code#hidden' 
# In the 2018 version of the SDG survey in SuAVE, this field is 'ISO3166N3#hidden' 
# To switch, comment one of the two lines below, uncomment the other

#                 curr_row = old_df.loc[old_df['UN Code#hidden'] == geo_code]
                curr_row = old_df.loc[old_df['ISO3166N3#hidden'] == geo_code]
                if (not curr_row.empty):
                    
                    curr_index = curr_row.index[0]
                
                    old_df.at[curr_index, new_col_name] = value
                
                

## 8. Save the new version of CSV file, and give a name to new survey

In [ ]:
# new file name
new_file =  csv_file[:-4] + '_v1.csv'
new_file_path = os.path.join(absolutePath, new_file)
old_df.to_csv(os.path.join(absolutePath, new_file), index=False)



In [ ]:
with pd.option_context("display.max_columns", None):
    display(old_df.drop(['geometry'],axis=1))


In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

## 9. Generate the survey and create survey URL

In [ ]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file_path, "rb")}
upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)

